# Vertex pipelines

**Learning Objectives:**
  1. Use `google_cloud_pipeline_components` to train custom model
  1. TODO

## Overview

This notebook shows how to use the components defined in [`google_cloud_pipeline_components`](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud) in conjunction with an experimental `run_as_aiplatform_custom_job` method, to build a [Vertex Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines) workflow that trains a [custom model](https://cloud.google.com/vertex-ai/docs/training/containers-overview), uploads the model, creates an *endpoint*, and deploys the model to the endpoint. 


### Objective

In this example, you'll learn how to use components from `google_cloud_pipeline_components` to:
- upload a model
- create an *endpoint*
- deploy the trained model to the endpoint for serving

In addition, you'll use the `kfp.v2.google.experimental.run_as_aiplatform_custom_job` method to train a custom model.

The components are [documented here](https://google-cloud-pipeline-components.readthedocs.io/en/latest/google_cloud_pipeline_components.aiplatform.html#module-google_cloud_pipeline_components.aiplatform).
(From that page, see also the `CustomPythonPackageTrainingJobRunOp` and `CustomContainerTrainingJobRunOp` components, which similarly run 'custom' training, but as with the related `google.cloud.aiplatform.CustomContainerTrainingJob` and `google.cloud.aiplatform.CustomPythonPackageTrainingJob` methods from the [Vertex AI SDK](https://googleapis.dev/python/aiplatform/latest/aiplatform.html), also upload the trained model).

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI Training and Serving
* Cloud Storage

Learn about pricing for [Vertex AI](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Google Cloud Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages


In [2]:
!pip3 install --user google-cloud-aiplatform --upgrade
!pip3 install --user kfp google-cloud-pipeline-components --upgrade

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

Check the versions of the packages you installed.  The KFP SDK version should be >=1.6.

In [4]:
!python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"

KFP SDK version: 1.6.4


## Before you begin

This notebook does not require a GPU runtime.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [27]:
import os

PROJECT_ID = "munn-sandbox"
BUCKET = "munn-sandbox"

os.environ['PROJECT_ID'] = PROJECT_ID
os.environ['BUCKET'] = BUCKET

In [7]:
!echo {PROJECT_ID}

munn-sandbox


#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [26]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
os.environ['TIMESTAMP'] = TIMESTAMP

### Authenticate your Google Cloud account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

### Create a Cloud Storage bucket as necessary

You will need a Cloud Storage bucket for this example.  If you don't have one that you want to use, you can make one now.


Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Vertex AI services are
available](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions). You may
not use a Multi-Regional Storage bucket for training with Vertex AI.

In [13]:
BUCKET_NAME = f"gs://{PROJECT_ID}/vertex_pipelines"
REGION = "us-central1"

os.environ['BUCKET_NAME'] = BUCKET_NAME
os.environ['REGION'] = REGION

print(BUCKET_NAME)
print(REGION)

gs://munn-sandbox/vertex_pipelines
us-central1


**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [16]:
! gsutil ls -al $BUCKET_NAME

CommandException: One or more URLs matched no objects.


### Import libraries and define constants

Define some constants. 

In [17]:
# PATH=%env PATH
# %env PATH={PATH}:/home/jupyter/.local/bin

# USER = "munn@google.com"  # <---CHANGE THIS
# PIPELINE_ROOT = "{}/pipeline_root/{}".format(BUCKET_NAME, USER)

# PIPELINE_ROOT

Do some imports:

In [18]:
import kfp
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2 import compiler
from kfp.v2.dsl import component
from kfp.v2.google import experimental
from kfp.v2.google.client import AIPlatformClient

## Define a pipeline that uses the components


Set some variables that will be used in constructing the args passed to the custom training job and setting pipeline params.

In [19]:
# # create args list for trainer

# hp_dict: str = '{"num_hidden_layers": 3, "hidden_size": 32, "learning_rate": 0.01, "epochs": 1, "steps_per_epoch": -1}'
# data_dir: str = "gs://aju-dev-demos-codelabs/bikes_weather/"
# TRAINER_ARGS = ["--data-dir", data_dir, "--hptune-dict", hp_dict]

# # create working dir to pass to job spec
# import time

# ts = int(time.time())
# WORKING_DIR = f"{PIPELINE_ROOT}/{ts}"

# MODEL_DISPLAY_NAME = f"train_deploy{ts}"
# print(TRAINER_ARGS, WORKING_DIR, MODEL_DISPLAY_NAME)

In [22]:
!gsutil ls gs://munn-sandbox/taxifare/data/taxi-train*

gs://munn-sandbox/taxifare/data/taxi-train-000000000000.csv


In [23]:
!gsutil ls gs://munn-sandbox/taxifare/data/taxi-valid*

gs://munn-sandbox/taxifare/data/taxi-valid-000000000000.csv


In [24]:
!gsutil ls gs://munn-sandbox/taxifare/data/taxi-test*

CommandException: One or more URLs matched no objects.


Next, you define a component with which the custom training job is run.  For this example, this component doesn't do anything (but run a print statement).

In [25]:
@component
def training_op(input1: str):
    print("training task: {}".format(input1))

Now, you define the pipeline.  

The `experimental.run_as_aiplatform_custom_job` method takes as args the component defined above, and the list of `worker_pool_specs`— in this case  one— with which the custom training job is configured. 
See [full function code here](https://github.com/kubeflow/pipelines/blob/master/sdk/python/kfp/v2/google/experimental/custom_job.py)

Then, [`google_cloud_pipeline_components`](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud) components are used to define the rest of the pipeline: upload the model, create an endpoint, and deploy the model to the endpoint. (While not shown in this example, the model deploy will create an endpoint if one is not provided.)

In [34]:
# Output directory and job_name
OUTDIR=f"gs://{BUCKET}/taxifare/trained_model_{TIMESTAMP}"
JOB_NAME=f"taxifare_{TIMESTAMP}"

PYTHON_PACKAGE_URIS=f"gs://{BUCKET}/taxifare/taxifare_trainer-0.1.tar.gz"
MACHINE_TYPE="n1-standard-4"
REPLICA_COUNT=1
PYTHON_PACKAGE_EXECUTOR_IMAGE_URI="us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-3:latest"
PYTHON_MODULE="trainer.task"

# Model and training hyperparameters
BATCH_SIZE=50
NUM_EXAMPLES_TO_TRAIN_ON=5000
NUM_EVALS=100
NBUCKETS=10
LR=0.001
NNSIZE="32 8"

# GCS paths
GCS_PROJECT_PATH=f"gs://{BUCKET}/taxifare"
DATA_PATH=f"{GCS_PROJECT_PATH}/data"
TRAIN_DATA_PATH=f"{DATA_PATH}/taxi-train*"
EVAL_DATA_PATH=f"{DATA_PATH}/taxi-valid*"

WORKER_POOL_SPEC=f"machine-type={MACHINE_TYPE},\
replica-count={REPLICA_COUNT},\
executor-image-uri={PYTHON_PACKAGE_EXECUTOR_IMAGE_URI},\
python-module={PYTHON_MODULE}"

ARGS=f"--eval_data_path={EVAL_DATA_PATH},\
--output_dir={OUTDIR},\
--train_data_path={TRAIN_DATA_PATH},\
--batch_size={BATCH_SIZE},\
--num_examples_to_train_on={NUM_EXAMPLES_TO_TRAIN_ON},\
--num_evals={NUM_EVALS},\
--nbuckets={NBUCKETS},\
--lr={LR},\
--nnsize={NNSIZE}"


In [36]:
print(ARGS)
print(WORKER_POOL_SPEC)

--eval_data_path=gs://munn-sandbox/taxifare/data/taxi-valid*,--output_dir=gs://munn-sandbox/taxifare/trained_model_20210707185813,--train_data_path=gs://munn-sandbox/taxifare/data/taxi-train*,--batch_size=50,--num_examples_to_train_on=5000,--num_evals=100,--nbuckets=10,--lr=0.001,--nnsize=32 8
machine-type=n1-standard-4,replica-count=1,executor-image-uri=us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-3:latest,python-module=trainer.task


In [69]:
@kfp.dsl.pipeline(name="train-endpoint-deploy-" + TIMESTAMP)
def pipeline(
    project: str = PROJECT_ID,
    model_display_name: str = JOB_NAME,
    serving_container_image_uri: str = "us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-3:latest",
):
    train_task = training_op("taxifare model training")
    experimental.run_as_aiplatform_custom_job(
        train_task,
        display_name=JOB_NAME,
#         replica_count=REPLICA_COUNT,
#         machine_type=MACHINE_TYPE,
        worker_pool_specs=[
            {
                "pythonPackageSpec": {
                    "executor_image_uri": PYTHON_PACKAGE_EXECUTOR_IMAGE_URI,
                    "package_uris": [PYTHON_PACKAGE_URIS],
                    "python_module": PYTHON_MODULE,
                    "args": [
                        f"--eval_data_path={EVAL_DATA_PATH}",
                        f"--output_dir={OUTDIR}",
                        f"--train_data_path={TRAIN_DATA_PATH}",
                        f"--batch_size={BATCH_SIZE}",
                        f"--num_examples_to_train_on={NUM_EXAMPLES_TO_TRAIN_ON}",
                        f"--num_evals={NUM_EVALS}",
                        f"--nbuckets={NBUCKETS}",
                        f"--lr={LR}",
                        f"--nnsize={NNSIZE}"
                    ],                    
                },
                "replica_count": f"{REPLICA_COUNT}",
                "machineSpec": {
                    "machineType": f"{MACHINE_TYPE}",
#                     "accelerator_type": aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
#                     "accelerator_count": 2,
                },                
            }
        ],
    )

    model_upload_op = gcc_aip.ModelUploadOp(
        project=project,
        display_name=model_display_name,
        artifact_uri=f"gs://{BUCKET}/vertex_pipelines/{TIMESTAMP}",
        serving_container_image_uri=serving_container_image_uri,
        serving_container_environment_variables={"NOT_USED": "NO_VALUE"},
    )
    model_upload_op.after(train_task)

    endpoint_create_op = gcc_aip.EndpointCreateOp(
        project=project,
        display_name="pipelines-created-endpoint",
    )

    model_deploy_op = gcc_aip.ModelDeployOp(  # noqa: F841
        project=project,
        endpoint=endpoint_create_op.outputs["endpoint"],
        model=model_upload_op.outputs["model"],
        deployed_model_display_name=model_display_name,
        machine_type="n1-standard-4",
    )

## Compile and run the pipeline

Now, you're ready to compile the pipeline:

In [70]:
from kfp.v2 import compiler  # noqa: F811

if not os.path.isdir("vertex_pipelines"):
    os.mkdir("vertex_pipelines")

compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="./vertex_pipelines/train_upload_deploy.json"
)

The pipeline compilation generates the `train_upload_deploy.json` job spec file.

Next, instantiate an API client object:

In [71]:
from kfp.v2.google.client import AIPlatformClient  # noqa: F811

api_client = AIPlatformClient(
    project_id=PROJECT_ID,
    region=REGION,
)

Then, you run the defined pipeline like this: 

In [72]:
response = api_client.create_run_from_job_spec(
    "./vertex_pipelines/train_upload_deploy.json",
    pipeline_root=f"gs://{BUCKET}/vertex_pipelines",
    parameter_values={"project": PROJECT_ID},
)

Click on the generated link to see your run in the Cloud Console.  It should look something like this:

<a href="https://storage.googleapis.com/amy-jo/images/mp/train_endpoint_deploy.png" target="_blank"><img src="https://storage.googleapis.com/amy-jo/images/mp/train_endpoint_deploy.png" width="75%"/></a>


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:
- Delete Cloud Storage objects that were created.  Uncomment and run the command in the cell below **only if you are not using the `PIPELINE_ROOT` path for any other purpose**.
- Delete your deployed model: first, undeploy it from its *endpoint*, then delete the model and endpoint.


In [ ]:
# Warning: this command will delete ALL Cloud Storage objects under the PIPELINE_ROOT path.
# ! gsutil -m rm -r $PIPELINE_ROOT